In [ ]:
import glob
import keras
import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.models import Sequential
from tensorflow import data as tf_data
from keras.layers import Dense, Dropout, GlobalAveragePooling1D, Input

In [ ]:
def parse_dataset():
    train_points = []
    train_labels = []
    class_map = {}
    folders = glob.glob("/kaggle/input/sign-points/dataset/")
    
    letters = "ABCDEFGHIKLMNOPQRSTUVWXY"
    
    for i, letter in enumerate(letters):
        print(f"Processing class: {letter} {i}")
        
        train_directory = f"/kaggle/input/sign-points/dataset/{letter}"
        
        class_map[i] = letter
        train_files = glob.glob(train_directory + "/*")
        
        for train_file in train_files:
            train_points.append(np.load(train_file))
            train_labels.append(i)
            
        print(len(train_points))
        print(len(train_labels))

    return (
        np.array(train_points),
        np.array(train_labels),
        class_map,
    )

In [ ]:
NUM_CLASSES = 24
BATCH_SIZE = 64

train_points, train_labels, CLASS_MAP = parse_dataset()

In [ ]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
train_dataset_size = int(len(dataset) * train_size)

dataset = dataset.shuffle(len(train_points))

train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [ ]:
# Input layer
inputs = keras.Input(shape=(21, 3))
x = Dense(128, activation='relu')(inputs)

# Hidden layers
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = GlobalAveragePooling1D()(x)

# Output layer for 26 ASL alphabets
outputs = Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="SLR")
model.summary()

In [ ]:
model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])
history = model.fit(train_dataset, epochs=100, validation_data=validation_dataset)

In [ ]:
model.save("model.keras")

In [ ]:
# Extracting the history of training and validation accuracy and loss
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Creating plots
plt.style.use('seaborn-whitegrid') # Use a modern style
colors = ['#377eb8', '#ff7f00']  # A set of colors: blue and orange
linestyles = ['-', '--']  # Solid and dashed lines

fig = plt.figure(figsize=(12, 6))
fig.patch.set_facecolor('#f7f7f7')

# Plotting training and validation accuracy
plt.subplot(1, 2, 1)
plt.title('Training and Validation Accuracy', fontsize=20)
plt.plot(epochs, acc, 'r', label='Training Accuracy', linestyle=linestyles[0], color=colors[0], linewidth=2)
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy", linestyle=linestyles[1], color=colors[1], linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True, linestyle='--', linewidth=0.2)

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.title('Training and Validation Loss', fontsize=20)
plt.plot(epochs, loss, 'r', label='Training Loss', linestyle=linestyles[0], color=colors[0], linewidth=2)
plt.plot(epochs, val_loss, 'b', label="Validation Loss", linestyle=linestyles[1], color=colors[1], linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.grid(True, linestyle='--', linewidth=0.2)

plt.savefig("training-and-validation-accuracy-and-loss.png")
plt.show()